In [52]:
#%%
from IPython import get_ipython
get_ipython().magic('reset -sf')

import sys
 
import os
import pandas as pd
import numpy as np
import pickle

C:\Users\josepht\AppData\Local\Temp\ipykernel_10392\2684249865.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [53]:
# Set relevant paths

current_dir = os.getcwd()  # location of files with code

code_dir = current_dir 
Output_dir = code_dir + '\\..\\' + "Outputs" # location of outputs generated by running RWA
analysis_dir = code_dir + '\\..\\' + "analysis" # location to save table for analysis

In [54]:
## Get EQB Personal Data
'''
file_path_eqb_pers=Output_dir + "\\eqb_rwa_original.xlsx"
sheet_name="RWA_Data"
df_rw_eqb_pers=pd.read_excel(file_path_eqb_pers, sheet_name=sheet_name, engine='openpyxl')
'''
file_path_eqb_pers=Output_dir + '\\SFR_rwa_2022_2024.csv'
df_rw_eqb_pers=pd.read_csv(file_path_eqb_pers)

df_rw_eqb_pers.shape

C:\Users\josepht\AppData\Local\Temp\ipykernel_10392\774711778.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rw_eqb_pers=pd.read_csv(file_path_eqb_pers)


(111459, 38)

In [55]:
df_rw_eqb_pers.head()

,Loan_Number,Insured_class,EAD,Advance_Amount,Years_to_maturity,corr_uninsured,rwa_airb,RWA_standardized,SL_Date,CalibratedPD,...,OSFI_LGD_floor,Final_LGD,deductible_amount,insured_PMI_ratio,Maturity_adj,corr_insured,risk_weight_uninsured,risk_weight_insured,RWA_new,Reporting_Date
0,100031,CMHC,154095.29,185600.68,2.253251,0.15,0.290595,0.000,2024-12-31,0.003622,...,0.1,0.1100,18560.068,0.879555,0.388207,0.239401,0.000000,0.001332,2565.453743,2024-12-31
1,100103,CMHC,91082.35,178107.95,2.414784,0.15,0.101277,0.000,2024-12-31,0.002228,...,0.1,0.1100,17810.795,0.804454,0.388207,0.239401,0.000000,0.001424,1621.546869,2024-12-31
2,100337,CMHC,196700.48,270155.62,1.749487,0.15,0.218717,0.000,2024-12-31,0.002228,...,0.1,0.1100,27015.562,0.862656,0.388207,0.239401,0.000000,0.001044,2566.491489,2024-12-31
3,100360,Uninsured,71475.09,149738.22,1.809719,0.15,13288.668913,14295.018,2024-12-31,0.007377,...,0.1,0.1821,14973.822,0.790503,0.150122,NaN,0.014873,0.000000,13288.085142,2024-12-31
4,100392,CMHC,112436.71,153218.68,1.749487,0.15,0.623504,0.000,2024-12-31,0.002228,...,0.1,0.1100,15321.868,0.863729,0.388207,0.239401,0.000000,0.001044,1467.041968,2024-12-31


In [56]:
df_rw_eqb_pers['SL_Date'].value_counts()

2022-12-31    58453
2024-12-31    53006
Name: SL_Date, dtype: int64

In [57]:
print(sorted(df_rw_eqb_pers.columns))

['AddOn', 'Advance_Amount', 'Alt_Prime_Indicator', 'Base_Line_LGD', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LoanNumber', 'Loan_Number', 'MRS_Bin_LGD', 'MRS_Bin_PD', 'Maturity_adj', 'OSFI_LGD_floor', 'Occupancy', 'Pre_final_LGD', 'RWA_new', 'RWA_standardized', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Reporting_Date', 'SL_Date', 'Segment_Avg_LGD', 'Years_to_maturity', 'corr_insured', 'corr_uninsured', 'deductible_amount', 'insured_PMI_ratio', 'risk_weight_insured', 'risk_weight_uninsured', 'rwa_airb']


In [41]:
### Add relevant columns for RWA reporting
df_wip=df_rw_eqb_pers.copy()
# EQB_Balance and Total_Balance
df_wip['Total_Balance']=df_wip['RemainingPrincipal_Tot_Exp']
df_wip['EQB_Balance']=df_wip['RemainingPrincipal_Excl_Partner']

# EAD_MM, Balance_MM, Limit_MM, 
#unit=1000000 # report exposure in millions

#EAD_Weighted_Final_PD, EAD_Weighted_pre_floor_LGD, EAD_Weighted_Final_LGD
df_wip['EAD_Weighted_Final_PD']=df_wip['CalibratedPD']*df_wip['EAD']
df_wip['EAD_Weighted_pre_floor_LGD']=df_wip['Pre_final_LGD']*df_wip['EAD']
df_wip['EAD_Weighted_Final_LGD']=df_wip['Final_LGD']*df_wip['EAD']


In [42]:
# Create Min and Max SL_Date
df_wip['Min_SL_Date'] = df_wip.groupby('Loan_Number')['SL_Date'].transform('min')
df_wip['Max_SL_Date'] = df_wip.groupby('Loan_Number')['SL_Date'].transform('max')


In [43]:
# create driver to indicate if loans in (both 22 and 24), 24 only and 22 only
cond_both=(df_wip['Min_SL_Date'] < df_wip['Max_SL_Date'])
df_wip.loc[cond_both,'Loan_Snapshots']="2024 & 2022"

cond_24=(df_wip['Min_SL_Date']=="2024-12-31")
df_wip.loc[cond_24,'Loan_Snapshots']="2024 only"

cond_22=(df_wip['Max_SL_Date']=="2022-12-31")
df_wip.loc[cond_22,'Loan_Snapshots']="2022 only"

print(sorted(df_wip.columns))

['AddOn', 'Advance_Amount', 'Alt_Prime_Indicator', 'Base_Line_LGD', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_Weighted_Final_LGD', 'EAD_Weighted_Final_PD', 'EAD_Weighted_pre_floor_LGD', 'EQB_Balance', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LoanNumber', 'Loan_Number', 'Loan_Snapshots', 'MRS_Bin_LGD', 'MRS_Bin_PD', 'Maturity_adj', 'Max_SL_Date', 'Min_SL_Date', 'OSFI_LGD_floor', 'Occupancy', 'Pre_final_LGD', 'RWA_new', 'RWA_standardized', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Reporting_Date', 'SL_Date', 'Segment_Avg_LGD', 'Total_Balance', 'Years_to_maturity', 'corr_insured', 'corr_uninsured', 'deductible_amount', 'insured_PMI_ratio', 'risk_weight_insured', 'risk_weight_uninsured', 'rwa_airb']


In [44]:
print(sorted(df_wip.columns))

['AddOn', 'Advance_Amount', 'Alt_Prime_Indicator', 'Base_Line_LGD', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_Weighted_Final_LGD', 'EAD_Weighted_Final_PD', 'EAD_Weighted_pre_floor_LGD', 'EQB_Balance', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LoanNumber', 'Loan_Number', 'Loan_Snapshots', 'MRS_Bin_LGD', 'MRS_Bin_PD', 'Maturity_adj', 'Max_SL_Date', 'Min_SL_Date', 'OSFI_LGD_floor', 'Occupancy', 'Pre_final_LGD', 'RWA_new', 'RWA_standardized', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Reporting_Date', 'SL_Date', 'Segment_Avg_LGD', 'Total_Balance', 'Years_to_maturity', 'corr_insured', 'corr_uninsured', 'deductible_amount', 'insured_PMI_ratio', 'risk_weight_insured', 'risk_weight_uninsured', 'rwa_airb']


In [46]:
crosstab_LGD_snap = pd.crosstab(df_wip['SL_Date'], df_wip['Loan_Snapshots'])
print(crosstab_LGD_snap)

Loan_Snapshots  2022 only  2024 & 2022  2024 only
SL_Date                                          
2022-12-31          19745        38708          0
2024-12-31              0        38708      14298


In [47]:
pd.crosstab(df_wip['MRS_Bin_LGD'], df_wip['SL_Date'])

SL_Date,2022-12-31,2024-12-31
MRS_Bin_LGD,,
1,25497,22377
2,19068,21216
3,6837,5276
4,7051,4137


In [50]:
# Save loan (1) in 2022 and not in 2024 and (2) in 2024 not in 2022 and share with FZ for inputs
df_22only=df_wip.loc[cond_22,['Loan_Number','SL_Date']].copy()
df_24only=df_wip.loc[cond_24,['Loan_Number','SL_Date']].copy()

# write file to excel 
with pd.ExcelWriter(
   Output_dir + '\eqb_pers_single_snapshot_loans.xlsx') as writer:           
     df_22only.to_excel(writer, sheet_name="2022_only")
     df_24only.to_excel(writer, sheet_name="2024_only")


In [51]:
# write file to excel for pivot
with pd.ExcelWriter(
   analysis_dir + '\RWA_Analysis_Report_EQB_Pers_v2_2.xlsx') as writer:           
     df_wip.to_excel(writer, sheet_name="RWA_Data")